### xgboost baseline 87.625

In [1]:
import warnings
warnings.simplefilter("ignore")
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
train=pd.concat([
    pd.read_csv('data/train/SQL注入.csv'),
    pd.read_csv('data/train/XSS跨站脚本.csv'),
    pd.read_csv('data/train/命令执行.csv'),
    pd.read_csv('data/train/白.csv'),
    pd.read_csv('data/train/目录遍历.csv'),
    pd.read_csv('data/train/远程代码执行.csv'),
],axis=0).reset_index(drop=True)


test=pd.read_csv('data/test.csv')
test['lable'] = -1
data = pd.concat([train, test])
train.shape, test.shape, data.shape

((33219, 7), (4000, 7), (37219, 7))

In [3]:
from ua_parser import user_agent_parser
def agent_parser(user_agent):
    if user_agent != "Null":
        parse_dict = user_agent_parser.Parse(user_agent)
        info_list = {}
        user_agent_info = dict(zip(['user_agent_'  + i for i in parse_dict['user_agent'].keys()], parse_dict['user_agent'].values()))
        os_info = dict(zip(['os_'  + i for i in parse_dict['os'].keys()], parse_dict['os'].values()))
        device_info = dict(zip(['device_'  + i for i in parse_dict['device'].keys()], parse_dict['device'].values()))
        temp_dict = {**user_agent_info, **os_info, **device_info}
        # df = pd.DataFrame.from_dict(temp_dict, orient='index').T
        return temp_dict
    else:
        return pd.DataFrame()

data['user_agent'] = data['user_agent'].fillna("Null")
# temp_df = pd.DataFrame()
agent_list = []
for i in tqdm(data['user_agent']):
    agent_list.append(agent_parser(i))
agent_df = pd.DataFrame(agent_list).drop(['os_minor', 'os_patch', 'os_patch_minor'], axis=1).reset_index()
data['index'] = agent_df['index'].tolist()
data = pd.merge(left=data, right=agent_df, on='index', how='left')
data.drop(['index'], axis=1, inplace=True)
data.shape

100%|██████████| 37219/37219 [00:05<00:00, 6688.74it/s]


(37219, 16)

In [4]:
num_ua = ['user_agent_major', 'user_agent_minor', 'user_agent_patch']
for i in data.columns[:]:
    if i in num_ua:
        data[i] = data[i].fillna('None')
        data[i] = data[i].replace({"None": '-999', 'b13pre': '13'})
        data[i] = data[i].astype('int')
        # print(data[i].unique())
    else:
        data[i] = data[i].fillna("None")
    
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37219 entries, 0 to 37218
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 37219 non-null  int64 
 1   method             37219 non-null  object
 2   user_agent         37219 non-null  object
 3   url                37219 non-null  object
 4   refer              37219 non-null  object
 5   body               37219 non-null  object
 6   lable              37219 non-null  int64 
 7   user_agent_family  37219 non-null  object
 8   user_agent_major   37219 non-null  int64 
 9   user_agent_minor   37219 non-null  int64 
 10  user_agent_patch   37219 non-null  int64 
 11  os_family          37219 non-null  object
 12  os_major           37219 non-null  object
 13  device_family      37219 non-null  object
 14  device_brand       37219 non-null  object
 15  device_model       37219 non-null  object
dtypes: int64(5), object(11)
memory usage: 4.

In [5]:
# from sklearn.decomposition import TruncatedSVD
# from sklearn.feature_extraction.text import TfidfVectorizer

# def tfidf_embeding(cols_name, n_components=16, min_df=3, max_df=0.5, ngram_range=(2, 6)):
#     texts=data[cols_name].str[:512].values.tolist()
#     tf = TfidfVectorizer(min_df= min_df, max_df=max_df, analyzer = 'char_wb', ngram_range=ngram_range)
#     X = tf.fit_transform(texts)
#     svd = TruncatedSVD(n_components=n_components,
#                     random_state=42)
#     X_svd = svd.fit_transform(X)
#     df_tfidf = pd.DataFrame(X_svd)
#     df_tfidf.columns = [cols_name + f'_tfidf_{i}' for i in range(n_components)]
#     return df_tfidf

# df_tfidf = pd.DataFrame()
# texts_col = ['user_agent', 'url', 'body']

# user_agent_tfidf = tfidf_embeding('user_agent')
# for i in tqdm(texts_col):
#     df_tfidf = pd.concat([df_tfidf, tfidf_embeding(i)], axis=1)
# df_tfidf.to_csv("df_tfidf.csv", index=False)
df_tfidf = pd.read_csv("df_tfidf.csv")
data = pd.concat([data, df_tfidf], axis=1)
df_tfidf.shape, data.shape


((37219, 48), (37219, 64))

In [6]:
from sklearn.preprocessing import LabelEncoder
cat_cols = []
for i in data.columns:
    if data[i].dtype == 'object' and i not in ['url', 'body']:
    # if data[i].dtype == 'object':
        cat_cols.append(i)
print(cat_cols)
lbe = LabelEncoder()

for i in cat_cols:
    data[i] = lbe.fit_transform(data[i])
    data[i + '_couts'] = data.groupby(i)[i].transform('count')
train, test = data[:len(train)], data[len(train):]
data.shape    

['method', 'user_agent', 'refer', 'user_agent_family', 'os_family', 'os_major', 'device_family', 'device_brand', 'device_model']


(37219, 73)

In [7]:
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)

# #需要编码的特征
# # enc_list=['trademark_cn','brand_cn','make_cn','series','client_no','p1_census_register']
# enc_list = cat_cols

# #标签
# label='lable'
# for f in enc_list:
#     train[f + '_target_enc'] = 0
#     test[f + '_target_enc'] = 0
#     for i, (trn_idx, val_idx) in enumerate(skf.split(train,train['lable'])):
#         trn_x = train[[f, label]].iloc[trn_idx].reset_index(drop=True)
#         val_x = train[[f]].iloc[val_idx].reset_index(drop=True)
#         enc_df = trn_x.groupby(f, as_index=False)[label].agg({f + '_target_enc': 'mean'})
#         val_x = val_x.merge(enc_df, on=f, how='left')
#         test_x = test[[f]].merge(enc_df, on=f, how='left')
#         val_x[f + '_target_enc'] = val_x[f + '_target_enc'].fillna(train[label].mean())
#         test_x[f + '_target_enc'] = test_x[f + '_target_enc'].fillna(train[label].mean())
#         train.loc[val_idx, f + '_target_enc'] = val_x[f + '_target_enc'].values
#         test[f + '_target_enc'] += test_x[f + '_target_enc'].values / skf.n_splits


# train=train.drop(labels=cat_cols, axis=1)
# test=test.drop(labels=cat_cols, axis=1)

In [64]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb

def xgb_model(train, target, test, k):
    feats = [f for f in train.columns if f not in ['lable',  'url', 'body', 'id']]
    print("Current num of features:", len(feats))
    oof_probs = np.zeros((train.shape[0], 6))
    output_preds = 0
    offline_score = []
    feature_importance_df = pd.DataFrame()
    params = {
        'eta': 0.05,
        'max_depth': 8,
        'booster': 'gbtree',
        'objective': 'multi:softprob',
        'num_class': 6,
        'eval_metric': 'mlogloss',
        'nthread': 4,
        'subsample': 0.7,
        # 'min_child_weight': 3,
        'colsample_bytree': 0.7,
        'seed':623,
        'tree_method': 'gpu_hist',
        'gpu_id': 0
    }
    seeds = [623]
    for seed in seeds:
        folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        for i, (train_index, test_index) in enumerate(folds.split(train, target)):
            print(f"\n------------Fold{i}----------------\n")
            train_y, test_y = target.iloc[train_index], target.iloc[test_index]
            train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
            dtrain = xgb.DMatrix(train_X, label=train_y)
            dval = xgb.DMatrix(test_X, label=test_y)
            xgb_model = xgb.train(params, 
                                  dtrain, 
                                  num_boost_round=10000,
                                  evals = [(dval, 'eval')],
                                  early_stopping_rounds=100,
                                  verbose_eval=100
                                  )

            oof_probs[test_index] = xgb_model.predict(xgb.DMatrix(test_X[feats]),
                                                       iteration_range=(xgb_model.best_iteration-1,  xgb_model.best_iteration+1)) / len(
                seeds)
            # offline_score.append(xgb.best_score['valid_0']['mlogloss'])
            output_preds += xgb_model.predict(xgb.DMatrix(test[feats]),
                                        iteration_range=(xgb_model.best_iteration-1,  xgb_model.best_iteration+1)) / folds.n_splits / len(seeds)
            # feature importance
            # fold_importance_df = pd.DataFrame()
            # fold_importance_df["feature"] = feats
            # fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
            # fold_importance_df["fold"] = i + 1
            # feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
            
    # print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    # print('feature importance:')
    # print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(50))

    return output_preds, oof_probs

            
    
    

In [65]:
feature_names = list(filter(
    lambda x: x not in ['lable', 'id', 'url', 'body'], train.columns
    
))
print(len(feature_names))

print('开始XGB模型训练train')
xgb_preds, xgb_oof = xgb_model(train=train[feature_names],
                               target=train['lable'],
                               test=test[feature_names], k=5)

69
开始XGB模型训练train
Current num of features: 69

------------Fold0----------------

[0]	eval-mlogloss:1.66166
[100]	eval-mlogloss:0.08609
[200]	eval-mlogloss:0.06773
[299]	eval-mlogloss:0.06958

------------Fold1----------------

[0]	eval-mlogloss:1.66268
[100]	eval-mlogloss:0.08999
[200]	eval-mlogloss:0.07295
[300]	eval-mlogloss:0.07425
[330]	eval-mlogloss:0.07500

------------Fold2----------------

[0]	eval-mlogloss:1.66269
[100]	eval-mlogloss:0.08732
[200]	eval-mlogloss:0.06743
[300]	eval-mlogloss:0.06753
[341]	eval-mlogloss:0.06816

------------Fold3----------------

[0]	eval-mlogloss:1.66190
[100]	eval-mlogloss:0.08037
[200]	eval-mlogloss:0.06031
[300]	eval-mlogloss:0.05997
[361]	eval-mlogloss:0.06074

------------Fold4----------------

[0]	eval-mlogloss:1.66047
[100]	eval-mlogloss:0.08529
[200]	eval-mlogloss:0.06543
[300]	eval-mlogloss:0.06568
[325]	eval-mlogloss:0.06620


In [66]:
print(accuracy_score(train['lable'],np.argmax(xgb_oof,axis=1)))

0.6917426773834251


In [63]:
sub=pd.read_csv('data/submit_example.csv')
sub['predict']=np.argmax(xgb_preds,axis=1)
sub.to_csv('xgb_sub.csv',index=None)